# Внешние отраслевые источники

## https://b1.ru/b1-surveys

In [6]:
import re
import time
import hashlib

import requests
from bs4 import BeautifulSoup


def get_soup(link):
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    soup = None
    response = requests.get(link, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Ошибка запроса: {response.status_code}")
    return soup


URL = 'https://b1.ru/'
SURVEYS_URL = URL + 'b1-surveys/'


soup = get_soup(SURVEYS_URL)
pagination = soup.find('div', class_='pagination')
page_links = [a['href'] for a in pagination.find_all('a') if '?PAGEN_2=' in a['href']]
last_page = max([int(a.split('?PAGEN_2=')[-1]) for a in page_links])

In [ ]:
from tqdm import tqdm


analytics = []
for i in tqdm(range(1, last_page + 1)):
    soup = get_soup(SURVEYS_URL + '?PAGEN_2=' + str(i))
    time.sleep(1)
    analytics_block = {}
    for block in soup.find_all('div', class_=re.compile(r"^news-block__gridItem triangleWrap--hover")):
        if block:
            link = block['src-href']
            if not 'services' in link:
                analytics_block['link'] = link
                analytics_block['date'] = block.find('p', class_='news-block__date').text
                analytics_block['name'] = block.find('h2', class_='news-block__title').text
                analytics_block['name_en'] = link.strip().strip('/').split('/')[-1]
                
                time.sleep(1)
                analytics_soup = get_soup(URL + link)
                tags_block = analytics_soup.find('div', class_=re.compile(r"tezis-list-block__tags"))
                if tags_block:
                    analytics_block['tags'] = [a.text for a in tags_block.find_all('a', class_=re.compile(r"tag"))]
                else:
                    analytics_block['tags'] = []
                
                analytics_block['text'] = '\n'.join([re.sub(r"\n+", "\n", x.text) for x in analytics_soup.find('main').find('section').find_next_siblings()])
                analytics_block['pdf_links'] = [f'{URL}{a['href']}' for a in analytics_soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
                analytics_block['pdf_links'].append(f'{URL}local/assets/surveys/{analytics_block['name_en']}.pdf')
                
                pdfs_content = []
                pdfs_names = []
                for url in analytics_block['pdf_links']:
                    response = requests.get(url, stream=True)
                    if response.status_code == 200:
                        pdf_name = url.strip().strip('/').split('/')[-1]
                        pdf_content = response.content
                        if pdf_name not in pdfs_names:
                            pdfs_names.append(pdf_name)
                            pdfs_content.append({"name": pdf_name, "content": pdf_content})
                analytics_block['pdfs'] = pdfs_content
                analytics.append(analytics_block)

100%|██████████| 7/7 [14:37<00:00, 125.40s/it]


In [16]:
import pickle

with open('analytics.pkl', 'wb') as f:
    pickle.dump(analytics, f)

In [24]:
import PyPDF2
import pdfplumber
import tempfile


def extract_table(pdf_path, page_num, table_num):
    pdf = pdfplumber.open(pdf_path)
    table_page = pdf.pages[page_num]
    table = table_page.extract_tables()[table_num]
    return table


def table_converter(table):
    table_string = ''
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string

In [27]:
with tempfile.NamedTemporaryFile() as temp_file:
    temp_file.write(analytics[0]['pdfs'][0]['content'])
    temp_file_path = temp_file.name
    pdfReaded = PyPDF2.PdfReader(temp_file_path)

In [28]:
pdfReaded

In [21]:
pdfReaded = PyPDF2.PdfReader(analytics[0]['pdfs'][0]['content'])

AttributeError: 'bytes' object has no attribute 'seek'

In [ ]:
pdf_path = 'OFFER 3.pdf'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

        # Проверяем элементы на наличие изображений
        if isinstance(element, LTFigure):
            # Вырезаем изображение из PDF
            crop_image(element, pageObj)
            # Преобразуем обрезанный pdf в изображение
            convert_to_images('cropped_image.pdf')
            # Извлекаем текст из изображения
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Добавляем условное обозначение в списки текста и формата
            page_text.append('image')
            line_format.append('image')

        # Проверяем элементы на наличие таблиц
        if isinstance(element, LTRect):
            # Если первый прямоугольный элемент
            if first_element == True and (table_num+1) <= len(tables):
                # Находим ограничивающий прямоугольник таблицы
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Извлекаем информацию из таблицы
                table = extract_table(pdf_path, pagenum, table_num)
                # Преобразуем информацию таблицы в формат структурированной строки
                table_string = table_converter(table)
                # Добавляем строку таблицы в список
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Устанавливаем флаг True, чтобы избежать повторения содержимого
                table_extraction_flag = True
                # Преобразуем в другой элемент
                first_element = False
                # Добавляем условное обозначение в списки текста и формата
                page_text.append('table')
                line_format.append('table')

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            if element.y0 >= lower_side and element.y1 <= upper_side:
                pass
            elif not isinstance(page_elements[i+1][1], LTRect):
                table_extraction_flag = False
                first_element = True
                table_num+=1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
os.remove('cropped_image.pdf')
os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])
print(result)